# Evaluator Module

In [1]:
#| default_exp evaluator

In [2]:
#| export
import CodeCheckList
import pandas as pd

import CodeCheckList.utils as utils
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker
from CodeCheckList.predictor import Predictor

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#| export
class Evaluator:
    """Evaluator Module to perform all AST Evaluations"""
    def __init__(self, checkpoint: str, language):
        self.tokenizer = CodeTokenizer.from_pretrained(checkpoint, language)
        self.masker = Masker(self.tokenizer)
        self.predictor = Predictor.from_pretrained(checkpoint, self.tokenizer)

    def __call__(self, test_set, number_of_predictions: int):
        results_dict = self.evaluate_test_set(test_set, number_of_predictions)
        results_dataframe = pd.DataFrame([], columns=['ast_element', 'occurences', 'successful_predictions', 'failed_predictions', 'total_predictions', 'success_average', 'failure_average'])
        for result_index, result in enumerate(results_dict):
            results_dataframe.loc[len(results_dataframe.index)] = [self.tokenizer.node_types[result_index] ,result[0], result[1], result[2], result[3], result[4], result[5]]
        return results_dataframe
    
    def evaluate_test_set(self, test_set, number_of_predictions: int):
        results_dict = []
        for node_type in self.tokenizer.node_types:
            results_dict.append([0, [0 for i in range(0,number_of_predictions)], 
                                [0 for i in range(0,number_of_predictions)], 
                                [0 for i in range(0,number_of_predictions)], 
                                [0 for i in range(0,number_of_predictions)],
                                [0 for i in range(0,number_of_predictions)]])
        for sample_index, sample in enumerate(test_set):
            print('-------------evaluating sample:'+str(sample_index)+'---------------------')
            sample_results = self.evaluate_code_snippet(sample['whole_func_string'], number_of_predictions)
            for element_index, element_result in enumerate(sample_results):
                element_result_values = list(element_result.values())[0]
                if len(element_result_values) > 0:
                    results_dict[element_index][0] += element_result_values[0][0]
                    for prediction_number_index in range(0, number_of_predictions):
                        results_dict[element_index][1][prediction_number_index]+= (1 if element_result_values[prediction_number_index][2] else 0)
                        results_dict[element_index][2][prediction_number_index]+= (0 if element_result_values[prediction_number_index][2] else 1)
                        results_dict[element_index][3][prediction_number_index]= results_dict[element_index][1][prediction_number_index] + results_dict[element_index][2][prediction_number_index]
                        results_dict[element_index][4][prediction_number_index]= results_dict[element_index][1][prediction_number_index]/results_dict[element_index][3][prediction_number_index]
                        results_dict[element_index][5][prediction_number_index]= results_dict[element_index][2][prediction_number_index]/results_dict[element_index][3][prediction_number_index]
        return results_dict
        
    def evaluate_code_snippet(self, source_code: str, number_of_predictions: int):
        evaluation_results = []
        for node_type_idx, node_type in enumerate(self.tokenizer.node_types):
            evaluation_results.append({node_type: self.evaluate_node_type_on_snippet(source_code, node_type_idx, number_of_predictions)})
        return evaluation_results
            
    def evaluate_node_type_on_snippet(self, source_code: str, target_node_type_idx: int, number_of_predictions: int):
        results=[]
        source_code_nodes = []
        utils.find_nodes(self.tokenizer.parser.parse(bytes(source_code, "utf8")).root_node, 
            self.tokenizer.node_types[target_node_type_idx], source_code_nodes)
        if len(source_code_nodes) == 0:
            return results

        masked_code_encoding = self.masker(source_code, self.tokenizer(source_code), target_node_type_idx)
        predictions = self.predictor(masked_code_encoding, self.tokenizer.tokenizer(source_code, return_tensors='pt'), number_of_predictions)  
        for prediction_number in range(0, number_of_predictions):
            predicted_code = predictions[prediction_number]
            predicted_nodes = []
            if utils.is_balanced_snippet(predicted_code):

                predicted_code_tree = self.tokenizer.parser.parse(bytes(predicted_code, "utf8")).root_node
                source_code_tree = self.tokenizer.parser.parse(bytes(source_code, "utf8")).root_node

                predicted_code_types = utils.get_node_type_set(predicted_code_tree)
                source_code_types = utils.get_node_type_set(source_code_tree)

                jaccard_distance = utils.calculate_jaccard_distance(predicted_code_types, source_code_types)
                sorensen_dice_distance = utils.calculate_sorensen_dice_distance(predicted_code_types, source_code_types)
                
            
                utils.find_nodes(predicted_code_tree, self.tokenizer.node_types[target_node_type_idx], predicted_nodes)
            results.append([len(source_code_nodes), len(predicted_nodes), len(predicted_nodes)>=len(source_code_nodes)])
            
        return results


## Full Pipeline

### Download Grammar

In [4]:
from CodeCheckList import loader

"""define language"""
python_language = "python"

languages = [python_language]

loader.download_grammars(languages)

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/CodeCheckList/grammars


### Load Model

In [5]:

"""define the model checkpoint"""
checkpoint = "huggingface/CodeBERTa-small-v1"

### Create Modules

In [6]:
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker

#create code tokenizer 
bert_tokenizer = CodeTokenizer.from_pretrained(checkpoint, python_language)

#create code masker
code_masker = Masker(bert_tokenizer)

### Node Types

In [7]:
print(bert_tokenizer.node_types)

['import_statement', 'case_clause', 'elif', 'keyword_separator', 'yield', '>>=', '<', 'lambda_parameters', 'while', 'tuple_pattern', 'continue_statement', 'decorated_definition', 'dictionary_comprehension', '%', '>>', 'return', '~', 'import_from_statement', '|=', 'parameter', '@', ':=', '>', 'integer', 'in', 'default_parameter', 'finally', 'set', 'or', '}}', 'attribute', 'from', 'format_specifier', '*=', 'continue', 'assert', 'dictionary', '!=', 'format_expression', 'del', 'as', 'else', 'try', 'subscript', 'type_conversion', 'except', 'pair', ';', 'async', 'pass_statement', 'pass', '_simple_statement', 'break', '[', 'await', 'for_statement', ']', 'dictionary_splat', 'with_clause', '-', 'assignment', 'named_expression', '**=', 'match', '"', '**', 'match_statement', '^=', 'float', 'typed_parameter', '==', 'comment', 'elif_clause', 'slice', 'typed_default_parameter', 'while_statement', '//=', 'type', '*', 'exec_statement', 'list_comprehension', 'try_statement', 'print_statement', ',', '+=

### Encodings

In [8]:
"""example source code"""

code = "def multiply_numbers(a,b):\n    return a*b"
#code = "def scale(self, center=True, scale=True):\n        \"\"\"\nthe the\n\n\n                                                                                                                                                          _\n                     ____________=_=_===========________===______________________________==_____________________\n_______\n____\n\n___\n\n\n\n\n\n\n\n\n        return return)"
#code = "def hello_world(a,b):\n    print('hello world')"
#code = "def __ordered_values_by_indexes(self, data, inds): \"\"\" Return values (intensities) by indexes. Used for multiscale graph cut. data = [[0 1 1], [0 2 2], [0 2 2]] inds = [[0 1 2], [3 4 4], [5 4 4]] return: [0, 1, 1, 0, 2, 0] If the data are not consistent, it will take the maximal value \"\"\" # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
#code = "def __ordered_values_by_indexes(self, data, inds):  # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
target_node_type = "*"

#encoding 
source_code_encoding = bert_tokenizer(code)

#masking
masked_code_encoding = code_masker(code, bert_tokenizer(code), bert_tokenizer.node_types.index(target_node_type))

assert len(source_code_encoding['input_ids']) == len(masked_code_encoding['input_ids'])

#masked code
masked_code = bert_tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == bert_tokenizer.tokenizer.bos_token_id or 
            token_id == bert_tokenizer.tokenizer.eos_token_id else True, masked_code_encoding['input_ids'])))

print(masked_code)

def multiply_numbers(a,b):
    return a<mask>b


### Code Prediction

In [9]:
from CodeCheckList.predictor import Predictor

predictor = Predictor.from_pretrained(checkpoint, bert_tokenizer)
predictions = predictor(masked_code_encoding, bert_tokenizer.tokenizer(code, return_tensors='pt'), 5)

------------------Loading Model into GPU------------------


### Evaluation

In [10]:
import CodeCheckList.utils as utils

prediction_number = 0
print('------------- CODE -------------')
print(code)
print('\n---------- MASKED -------------')
print(masked_code)
print('\n--------- PREDICTED -----------')
predicted_code = predictions[prediction_number]
print(predicted_code)
print('\n--------- AST COMPARE -----------')
filtered_nodes = []
filtered_nodes_predict = []
utils.find_nodes(bert_tokenizer.parser.parse(bytes(code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes)
utils.find_nodes(bert_tokenizer.parser.parse(bytes(predicted_code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes_predict)
print(len(filtered_nodes))
print(len(filtered_nodes_predict))
#base the evaluation on size comparison

------------- CODE -------------
def multiply_numbers(a,b):
    return a*b

---------- MASKED -------------
def multiply_numbers(a,b):
    return a<mask>b

--------- PREDICTED -----------
def multiply_numbers(a,b):
    return a,b

--------- AST COMPARE -----------
1
0


## Testing

In [11]:
from datasets import load_dataset 
import CodeCheckList.utils as utils


evaluator = Evaluator(checkpoint, python_language)

max_token_number = bert_tokenizer.tokenizer.max_len_single_sentence
print(max_token_number)

test_set = load_dataset("code_search_net", split='test')
test_set = test_set.filter(lambda sample: True if sample['language']== python_language
            and len(sample['func_code_tokens']) <= max_token_number
            and len(bert_tokenizer.tokenizer(sample['whole_func_string'])['input_ids']) <= max_token_number else False, num_proc=1)

test_set = utils.get_random_sub_set_test_set(utils.get_test_sets(load_dataset("code_search_net", split='test'), "python", evaluator.tokenizer.tokenizer.max_len_single_sentence, evaluator.tokenizer), 2)

len(test_set)

------------------Loading Model into GPU------------------
510


No config specified, defaulting to: code_search_net/all
Found cached dataset code_search_net (/home/svelascodimate/.cache/huggingface/datasets/code_search_net/all/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)
Parameter 'function'=<function <lambda> at 0x7fef041cd5a0> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 101/101 [00:24<00:00,  4.05ba/s]
No config specified, defaulting to: code_search_net/all
Found cached dataset code_search_net (/home/svelascodimate/.cache/huggingface/datasets/code_search_net/all/1.0.0/80a244ab541c6b2125350b

2

In [12]:
number_of_predictions = 2
checkpoint = "huggingface/CodeBERTa-small-v1"
python_language = "python"

evaluator = Evaluator(checkpoint, python_language)

results_dataframe = evaluator(test_set, number_of_predictions)

results_dataframe.sort_values(by=['occurences'], ascending=False)


------------------Loading Model into GPU------------------
-------------evaluating sample:0---------------------
0.0
0.027777777777777776
0.0
0.0
0.0
0.027777777777777776
0.0
0.0
0.0
0.027777777777777776
0.0
0.0
0.027777777777777776
0.10256410256410256
0.0
0.0
0.027777777777777776
0.05405405405405406
0.0
0.0
0.027777777777777776
0.027777777777777776
0.0
0.05405405405405406
0.0
0.0
0.027777777777777776
0.027777777777777776
0.0
0.0
0.027777777777777776
0.07894736842105263
0.0
0.0
0.0
0.027777777777777776
0.027777777777777776
0.07894736842105263
0.027777777777777776
0.05405405405405406
0.0
0.0
0.027777777777777776
0.027777777777777776
0.0
0.10256410256410256
0.07894736842105263
0.027777777777777776
0.0
0.05405405405405406
0.0
0.027777777777777776
0.07894736842105263
0.027777777777777776
0.0
0.0
0.05405405405405406
0.125
0.0
0.0
0.0
0.05405405405405406
0.027777777777777776
0.05405405405405406
0.05405405405405406
0.125
0.0
0.10256410256410256
0.10256410256410256
0.0
-------------evaluating 

,ast_element,occurences,successful_predictions,failed_predictions,total_predictions,success_average,failure_average
127,identifier,60,"[1, 1]","[1, 1]","[2, 2]","[0.5, 0.5]","[0.5, 0.5]"
64,"""",18,"[1, 0]","[1, 2]","[2, 2]","[0.5, 0.0]","[0.5, 1.0]"
101,.,16,"[1, 0]","[1, 2]","[2, 2]","[0.5, 0.0]","[0.5, 1.0]"
112,),13,"[1, 1]","[1, 1]","[2, 2]","[0.5, 0.5]","[0.5, 0.5]"
122,(,13,"[1, 1]","[1, 1]","[2, 2]","[0.5, 0.5]","[0.5, 0.5]"
...,...,...,...,...,...,...,...
74,typed_default_parameter,0,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]"
75,while_statement,0,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]"
76,//=,0,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]"
77,type,0,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]"
